# Eigenportfolio Construction Using Principle Component Analysis
By Austin Hardt on 20 March 2019.

We will be constructing eigenportfolios using principle component analysis (PCA). The data I will be using is obtained from the IEX Finance api. Although there are other API's and data sources, I chose IEX because it is free and provides intraday data. Note that eigenportofolios are normally built using daily prices over years, not intraday prices over 10 days. This is an experiment.

This notebook is lenghty so I have layed out the contents below. In short, what we will be learning is: how to collect, clean, and prepare real-time intraday data, how to construct eigen-portfoilios using principle components, and how to develop a brownian motion model to project future stock prices.


## Contents  
  
**[1. Collect Data](#collectdata)**  
&nbsp;&nbsp; [1.1 Retrieving Data From IEX](#retrieve)  
&nbsp;&nbsp; [1.2 Obtaining & Storing Data](#obtain)  
&nbsp;&nbsp; [1.3 Cleaning Data](#clean)  
&nbsp;&nbsp; [1.4 Visualization](#visual)  
  
**[2. Preparing Data](#prepare)**  
&nbsp;&nbsp; [2.1 Introduce an Index](#index)   
&nbsp;&nbsp; [2.2 Compute Percent Returns](#returns)  
&nbsp;&nbsp; [2.3 Splitting Data](#split)  
  
**[3. Eigen-Portfolio Construction](#eigen)**  
&nbsp;&nbsp; [3.1 Principle Component Analysis](#pca)   
&nbsp;&nbsp; [3.2 Define Sharpe Ratio](#sharpe)  
&nbsp;&nbsp; [3.3 Building our Eigen-Portfolios](#building)  
&nbsp;&nbsp; [3.4 Eigen-Portfolio Visualization](#viz)
 
 **[4. Projecting Stock Prices](#project)**   
&nbsp;&nbsp; [4.1 Establish New Training Set](#establish)   
&nbsp;&nbsp; [4.2 Develop Geometric Brownian Motion (GBM) Model](#gbm)  
&nbsp;&nbsp; [4.3 Prepare our New Data](#new)  
&nbsp;&nbsp; [4.4 Construct New Eigen-Portfolios](#construct)  
&nbsp;&nbsp; [4.5 Visualize our New Portfolios](#vis)

  
    
We begin, of course, by importing the various libraries we will be using.

In [ ]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay # B for buisiness, not birth
from iexfinance.stocks import get_historical_intraday
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
from sklearn.decomposition import PCA

<a id="collectdata"> </a> 
# 1. Collect Data
 
We must first collect our data. If you already have a data source or collection method that produces clean & accurate data, skip to Chapter 2.

<a id="retrieve"> </a> 
### 1.1 Retrieving Data From IEX
I may have complicated the process of obtaining data from IEX, so if you decide to use a different api (e.g., quandl) or data source, skip to section 1.2. However, my code is written to produce a specific format of the data, which is neccessary for PCA.


In [ ]:
# obtain intraday 
def get_intraday(ticker, date):
    df = pd.DataFrame(get_historical_intraday(ticker, date))
    if df.empty or df['marketAverage'].isnull().any().any() : return None
    df['date'] = pd.to_datetime(df['date'] + ' ' + df['minute'])
    return df.set_index('date')['marketAverage']

# obtain data for multiple days
def get_multiple_days(ticker, period):
    intraday_data = []
    for day in reversed(range(period)):
        date = datetime.datetime.today() - BDay(day) - BDay(1) # TODO remove this extra day.
        intraday_data.append(get_intraday(ticker, date))
    df = pd.concat(intraday_data).rename(ticker)
    return df[df != -1].dropna()

# combine data for each stock into a dataframe
def merge_data(period):
    stocks, tickers = [], get_tickers()
    for ticker in tqdm(tickers):
        try: stocks.append(get_multiple_days(ticker, period))
        except: pass
    df = pd.concat(stocks, axis=1)
    return df

# returns a list of tickers.
def get_tickers():
    return list(pd.read_csv('tickers.csv')['Symbol'])

<a id="obtain"> </a> 
### 1.2 Obtaining & Storing Data
I have written the above functions in such a way that I collect the entirety of my dataframe into a single variable. However, since im using IEX and not an already prepared data file, this takes quite a while.

In [ ]:
%%time
data = merge_data(10)

So, lets store this dataframe as a csv file. This way (hopefully) we never have to run the above script again.

In [ ]:
data.to_csv('data_test.csv')

Easy. Now we just read our csv, and take a look.

In [ ]:
# reads csv
def read(filename):
    return pd.read_csv(filename, index_col='date') #make sure to specify the date is the index!

In [ ]:
df = read('data_test.csv')
print('Dimensions of our data: ', df.shape)
df.iloc[:5, :15]

<a id="clean"> </a> 
### 1.3 Cleaning Data

Notice the NaNs. There are a good bit of them throughout the data. We must take care of them before we even think about building a model. In this case, I dropped all stocks that did not have at least 97% good data, then filled some others using a rolling mean. Note that this is a dangerous thing to do, so be very thorough if you use this method.

In [ ]:
# cleans the data
def clean_data(df):
    df = df.dropna(axis=1, thresh = int(round(len(df.index) * 0.95))) # drops stocks without 95% good data.
    df = df.fillna(df.rolling(window=10, min_periods=1).mean(), axis=1) # rolling mean to fill isolated NaNs.
    df = df.dropna(axis=0)
    return df.dropna(axis=1)

In [ ]:
df = clean_data(df)
print('New dimensions: ', df.shape)

Although we lost a decent amount of stocks, its important to not be lenient when filling NaNs. Adding more and more 'false' data will decrease the validity of our model.

<a id="visual"> </a> 
### 1.4 Visualization (Optional)

Note that for most data science projects, visualizing the data is just as important as preparation or modelling. In fact, it should deserve its own chapter. However, since we all already know what stock market data looks like, and since there are tens of online tools that offer visualization better than that of matplotlib or seaborn, we wont spend much time on this.

In [ ]:
df.sample(1, axis=1).plot(figsize=(16,8), title="Price as a Function of Time for a Randomly Selected Stock")

<a id="prepare"> </a> 
# 2. Preparing Data

Now that we have collected our data and it is clean, we may begin preparing it for PCA.

<a id="index"> </a> 
### 2.1 Introduce an Index
Lets sort our columns alphabetically and introduce an index. Since these companies are drawn from different exchanges, we will need an index in order to compare our eigenportfolios with the market. I defined it to be the average price of all companies combined. Note this is definitely not how market indexes are calculated.

In [ ]:
# sort columns alphabetically
def sort_data(df):
    return df.reindex(sorted(df.columns), axis=1) 

# introduces market index
def add_index(df):
    df['INDEX'] = df.sum(axis=1) / len(df.columns) 
    return df

In [ ]:
df = add_index(sort_data(df))
print('Stock Prices Shape:', df.shape)
df.iloc[-5:, -15:] # the index is found at the end.

<a id="returns"> </a> 
### 2.2 Compute Percent Returns

We first initialize two empty matrices of the same dimensions as our dataframe. Then, we calculate percent returns and standardize them. Note the first row of the data will be NaN, so we will ignore (remove) it.

Now we standardize the percent returns. This is done by demeaning the returns and scaling by the standard deviation,

$$ Z = \frac{X - \mu}{\sigma},$$

where $X$, $\mu$, and $\sigma$ are respectively the percent returns, mean, and standard deviation. We set $Z$, the standardized values, to fill our other empty matrix.

In [ ]:
# computes percent returns
def compute_stock_returns(df):
    stock_returns = pd.DataFrame(data = np.zeros(shape = (len(df.index), df.shape[1])),
                                 columns = df.columns.values, index = df.index)
    normed_returns = stock_returns
    stock_returns = df.pct_change().dropna()
    return (stock_returns, standardize(stock_returns))

# standardizes the pct returns.
def standardize(stock_returns):
    return (stock_returns - stock_returns.mean()) / stock_returns.std()

In [ ]:
[stock_returns, normed_returns] = compute_stock_returns(df)
normed_returns.iloc[-5:, -10:]

<a id="split"> </a> 
### 2.3 Splitting Data

I chose to keep 90% of the data as the training set and the rest as the testing set. Note in finance the training set and testing set may be refered as 'in-sample' and 'out-of-sample'.

In [ ]:
# determine where to split
def where_to_split(df):
    split_where = int(df.shape[0] * 0.9)
    here = df[split_where - 1:split_where]
    return here.index[0]

# split the data
def split_data(df_returns):
    train = df_returns[df_returns.index <= where_to_split(df_returns)].copy()
    test = df_returns[df_returns.index > where_to_split(df_returns)].copy()
    return (train, test)

In [ ]:
[df_train, df_test] = split_data(normed_returns)
[df_raw_train, df_raw_test] = split_data(stock_returns)
print('Train dataset:', df_train.shape)
print('Test dataset:', df_test.shape)

In [ ]:
# all the above results are reduced to one line
[df_raw_train_tmp, df_raw_test_tmp] = split_data(compute_stock_returns(add_index(sort_data(clean_data(read('data.csv')))))[0])
# check if we get the same result this way.
if (df_raw_train_tmp.equals(df_raw_train) and df_raw_test_tmp.equals(df_raw_test)): print('Functions are Reproducible')

<a id="eigen"> </a> 
# 3. Eigen-Portfolio Construction

Now that our data is prepared, we may begin constructing our eigen-portfolios.
<a id="pca"> </a> 
### 3.1 Principle Component Analysis
We compute principle component analysis using our data. Then, we fix variance explained at some number and see what is the smallest number of components needed to explain this variance.

We begin by computing the covariance matrix of our standarized training data. We then fit our model to the covariance matrix.

In [ ]:
# computes covariance matrix and fits pca model to it.
def compute_pca(df_train):
    pca = None
    stock_tickers = df_train.columns.values[:-1]
    cov_matrix = pd.DataFrame(data=np.ones(shape=(len(stock_tickers), len(stock_tickers))), columns = stock_tickers)
    cov_matrix = df_train.loc[:, df_train.columns != 'INDEX'].cov()
    return (PCA().fit(cov_matrix), cov_matrix)

# prints how many components explain variance threshold, which I defined to be 80%
def explain_variance(pca, var_thresh=0.8):
    var_explained = np.cumsum(pca.explained_variance_ratio_)
    num_comp = np.where(np.logical_not(var_explained < var_thresh))[0][0] + 1
    print('{0} components explain {1}% of variance.'.format(num_comp, var_thresh * 100))


# plots the variance explained    
def plot_variance_explained(pca, cov_matrix):
    n_asset = int((1/10) * cov_matrix.shape[1]) + 1
    x_indx = np.arange(0, n_asset)
    fig, ax = plt.subplots()
    fig.set_size_inches(18, 6)
    rects = ax.bar(x_indx, pca.explained_variance_ratio_[:n_asset], 1.2) # bar width = 1.2
    ax.set_xticks(x_indx + 0.6) # xtick begins at end of bar
    ax.set_xticklabels(list(range(n_asset)), rotation=66.6)
    ax.set_title('Percent Variance Explained')
    ax.legend((rects[0],), ('percent variance explained by principal components',))
    plt.show()

In [ ]:
[pca, cov_matrix] = compute_pca(df_train)
explain_variance(pca)
plot_variance_explained(pca, cov_matrix)

Next, we preform a fit-transform on the covariance matrix

In [ ]:
def fit_transform_pca(pca, cov_matrix):
    return pca.fit_transform(cov_matrix)

In [ ]:
projected = fit_transform_pca(pca, cov_matrix)

<a id="sharpe"> </a> 
### 3.2 Define Sharpe Ratio

We define a sharpe metric in order to measure the preformance of our portfolios. The normal definition of the sharpe ratio describes preformance of annualized returns, but, in this experiment, we are using intraday data in a period of only ten days. Therefore, we will define a psuedo-sharpe-ratio to describe hourly preformance.

In [ ]:
def sharpe_ratio(returns, periods=60): # working with minutely data, 60 min in hour.
    n_hours = (returns.shape[0] / periods)
    hourly_return = np.power(np.prod(1 + returns),(1 / n_hours)) - 1
    hourly_vol = returns.std() * np.sqrt(periods)
    return hourly_return / hourly_vol

<a id="building"> </a> 
### 3.3 Building our Eigen-Portfolios

We define an empty dictionary where we will store our portfolios. 

In [ ]:
eigen_portfolios = {}

The respective amounts invested in each stock, or the weights $w$, are defined as


$$w_i^{(j)} = \frac{v_i^{(j)}}{\overline\sigma_i},$$


where $v$ is the set of all eigenvectors for our projection space. Specifically, $v$ are the principle components. Note that $j$ is the index of the eigen-portfolio and $i$ is representative of the stocks contained in $j$.

The eigen-portfolio returns, $F$, for some duration of time, $k$, are therefore

$$ F_{jk} = \sum_{i}^{N} w_i^{(j)}R_{ik},$$

where $R$ is stock $i$'s return data. 

Note that, normally, eigen portfolios contain negative weights (they still add up to 1). These negative weights pertain to shorts, where you'll need to borrow money. In this experiment, I negate negative weights and define my portfolios to consist of only 5 stocks. I am developing this program in the same sense as to why I am using intraday data: I want to know if these methods are viable for high frequency algorithmic trading.

In [ ]:
# builds eigen-portfolios and appends them to our dictionary.
# Note that in pca.components_, each row is a principle component
# and that each column represents each feature.
def construct_eigen_portfolios(eigen_portfolios, pca, df_raw_test, stock_tickers, n_portfolios=200):
    sharpe_metric = np.array([0.] * n_portfolios)
    for ix in range(n_portfolios):
        eigen_portfolio = pd.DataFrame(data={'Weights': weights(pca.components_, ix).squeeze()*100}, index = stock_tickers)
        eigen_portfolio.sort_values(by=['Weights'], ascending=False, inplace=True)
        eigen_portfolio = adjust_weights(eigen_portfolio)
        portfolio_stocks = eigen_portfolio['Weights'].index
        raw_test = df_raw_test[portfolio_stocks.values]
        eigen_returns = pd.Series(returns(eigen_portfolio, raw_test).squeeze(), index=raw_test.index)
        sharpe_metric[ix] = sharpe_ratio(eigen_returns)
        eigen_portfolios.update({'Portfolio {0}'.format(ix): {'Sharpe Ratio': sharpe_metric[ix],
                                                              'Weights': eigen_portfolio['Weights'],
                                                              'Returns': eigen_returns}})
    return eigen_portfolios

# computes weights for portfolio
def weights(pcs, ix):
    return pcs[:, ix] / sum(pcs[:, ix]) #normalize to 1

# computes the returns for a given portfolio
def returns(eigen_portfolio, df_raw_test):
    return np.dot(df_raw_test.loc[:, eigen_portfolio.index], eigen_portfolio / 100)

# removes negative weights
def adjust_weights(eigen_portfolio):
    adjusted_weights = eigen_portfolio['Weights']
    adjusted_weights = adjusted_weights.nlargest(10) # I only want 5 stocks in my portfolio
    adjusted_weights = adjusted_weights / adjusted_weights.sum() * 100
    eigen_portfolio['Weights'] = adjusted_weights
    return eigen_portfolio.dropna()

In [ ]:
eigen_portfolios = construct_eigen_portfolios(eigen_portfolios, pca, df_raw_test, df.columns.values[:-1])

<a id="viz"> </a> 
### 3.4 Eigen-Portfolio Visualization

We now visualize our portfolios and gain insights as to which ones preform the best.

In [ ]:
# collect sharpe ratios for all portfolios
def sharpes(eigen_portfolios):
    sharpes = pd.DataFrame(eigen_portfolios).transpose()['Sharpe Ratio'].rename_axis('Portfolios')
    return pd.DataFrame(sharpes.sort_values(ascending=False))

# visualize distribution of shapre ratios for all portfolios
def plot_sharpes(sharpes):
    fig, ax = plt.subplots()
    fig.set_size_inches(16, 6)
    ax.plot(sharpes, linewidth=3)
    ax.set_title('Sharpe Ratio of Eigen-Portfolios')
    ax.set_ylabel('Sharpe ratio')
    ax.set_xlabel('Portfolios')
    plt.show()

In [ ]:
print('Average Sharpe Ratio:', sharpes(eigen_portfolios).mean().values[0])
plot_sharpes(sharpes(eigen_portfolios).sample(len(eigen_portfolios)).values)

In [ ]:
sharpes(eigen_portfolios).head(20)

These sharpe ratios are quite bad. I should note that before I decided to select only 5 stocks for my portfolio (that is, when these portfolios contained negative weights), they preformed better and displayed higher sharpe ratios — as one could imagine.

In [ ]:
sharpest_portfolio = sharpes(eigen_portfolios).index[0]

In [ ]:
# visualize portfolio weight distribution
def plot_portfolio_weights(eigen_portfolios, portfolio):
    stock_tickers = eigen_portfolios[portfolio]['Weights'].index
    eigen_portfolios[portfolio]['Weights'].plot(title='{0} weights'.format(portfolio), figsize=(12,6), 
                      xticks=range(len(stock_tickers)), rot=45, linewidth=3)

# compare portfolio preformance with market index
def plot_portfolio_vs_index(eigen_portfolios, portfolio, df_raw_test):
    df_plot = pd.DataFrame({'Portfolio': eigen_portfolios[portfolio]['Returns'],
                                                          'INDEX': df_raw_test['INDEX']}, index=df_raw_test.index)
    np.cumprod(df_plot + 1).plot(title='Returns of the market index vs. {0}'.format(portfolio),
                                 figsize=(12,6), linewidth=3)

In [ ]:
plot_portfolio_weights(eigen_portfolios, sharpest_portfolio)
plot_portfolio_vs_index(eigen_portfolios, sharpest_portfolio, df_raw_test)

<a id="project"> </a> 
# 4. Projecting Stock Prices

We have learned how to construct eigen-portfolios. Now we will take all of the available data as the training set, and predict stock prices, utlizing a geometric brownian motion (GBM) model, to use as our testing set.

<a id="establish"> </a> 
### 4.1 Establish New Training Set

We will take all of our previous data and use it as a training set.

In [ ]:
assets = add_index(sort_data(clean_data(read('data.csv'))))

That was easy enough, now for the hard part.

<a id="gbm"> </a> 
### 4.2 Develop Geometric Brownian Motion (GBM) Model

TODO: write up theory

In [ ]:
# generates new data
def gen_data(assets):
    tickers = list(assets.columns)
    dates = get_dates(assets)
    drift_vol = compute_drift_vol(assets)
    predictions = pd.DataFrame(np.zeros([len(dates), assets.shape[1]]), index=dates, columns=assets.columns)
    for ticker in tickers:
        price_paths = simulate(ticker, assets, dates)
        pred = pd.DataFrame(price_paths, columns=dates).sample(1).transpose() # taking one path
        predictions[ticker] = pred  # filling our predictions dataframe
    assets_preds = assets.append(predictions).rename_axis('date')
    assets_preds.index = assets_preds.index.astype(str)
    return assets_preds 

# simulates the model 25 times.
def simulate(ticker, assets, dates, n_simulations=25):
    drift_vol = compute_drift_vol(assets)
    price_paths = []
    for i in range(n_simulations):
        [mu, v] = drift_vol[ticker]
        price_path = generate_price_path(assets[ticker][-1], v, mu, dates)
        price_paths.append(price_path)
    return price_paths

# computes predicted prices for all future dates.
def generate_price_path(S, v, mu, dates):
    price_path = []
    for i in range(len(dates)):
        pred_price = generate_asset_price(S, v, mu)
        S = pred_price
        price_path.append(pred_price)
    return price_path

# computes predicted price.
def generate_asset_price(S,v,r,T=1/(30)):
    return S * np.exp((r - 0.5 * v**2) * T + v * np.sqrt(T) * np.random.normal(0,1))
 
# computes future dates (3 hours ahead of last known prices). 
def get_dates(assets):
    start = pd.to_datetime(assets.index[-1]) + pd.Timedelta(minutes=1)
    end = pd.to_datetime(assets.index[-1:]) + pd.Timedelta(minutes=30)
    return pd.date_range(start=start, end=end[0], freq='min')

# computes drift and volatility rates.
def compute_drift_vol(assets, window=10):       
    drift = ((assets[-window:].mean() - assets[:window].mean()) / assets[:window].mean()).rename('Drift') / 10
    vol = assets.rolling(window=window, min_periods=1).mean()[-window:].std().rename('Vol') / 10

    return pd.concat([drift, vol], axis=1).transpose()


In [ ]:
#plot Amazon predictions
pred = pd.DataFrame(simulate('SU', assets, get_dates(assets)), columns=get_dates(assets)).transpose()
pd.concat([assets['SU'], pred], axis=1).plot(figsize=(16, 3), legend=None)
#plot Apple predictions
pred = pd.DataFrame(simulate('WSM', assets, get_dates(assets)), columns=get_dates(assets)).transpose()
pd.concat([assets['WSM'], pred], axis=1).plot(figsize=(16, 3), legend=None)
#plot Nvidia
pred = pd.DataFrame(simulate('YEXT', assets, get_dates(assets)), columns=get_dates(assets)).transpose()
pd.concat([assets['YEXT'], pred], axis=1).plot(figsize=(16, 3), legend=None)

In [ ]:
%%time
assets_preds = gen_data(assets)

<a id="new"> </a> 
### 4.3 Prepare our New Data


Now we compute returns, split our data, and run principle component analysis.

In [ ]:
[asset_returns, returns_normed] = compute_stock_returns(assets_preds)
[train_raw, test_raw] = split_data(asset_returns)
[train, test] = split_data(returns_normed)
[pca, COV] = compute_pca(train)
explain_variance(pca)
plot_variance_explained(pca, COV)

<a id="construct"> </a> 
### 4.4 Construct New Eigen-Portfolios

Initialize empty dictionary.

In [ ]:
portfolios = {}

We construct our eigen portfolios.

In [ ]:
portfolios = construct_eigen_portfolios(portfolios, pca, test_raw, test_raw.columns.values[:-1])

<a id="vis"> </a> 
### 4.5 Visualize our New Portfolios

Lets see how these ones preformed.

In [ ]:
print('Average Sharpe Ratio:', sharpes(portfolios).mean().values[0])
plot_sharpes(sharpes(portfolios).sample(len(portfolios)).values)

In [ ]:
sharpes(portfolios).head(5)

Interesting, our portfolios outpreformed our previous ones. Could this be right?

In [ ]:
sharpest_portfolio = sharpes(portfolios).index[0]

In [ ]:
plot_portfolio_weights(portfolios, sharpest_portfolio)
plot_portfolio_vs_index(portfolios, sharpest_portfolio, test_raw)

Hmmm. We can practically see where our real data and GBM predicted data meet. It seems this method of eigen-portfolio construction exploits the GBM. Perhaps by reconstructing the GBM to be more random, we could achieve results that seem more realistic. Although, I do not think a GBM model should be utilized to project stock prices in this experiment. That being said, if one could approximate the drift and volatility coefficients in a precise manner, perhaps the model may be considered useful. 

Anyways, eigen-portfolio construction may be a viable method for portfolio selecting in the short term, but it will depend on market conditions, how much data you have, selection methods, and much more. In the long term, it is viable, and widely used. This program, with a few minor reconstructions, may be used to do just that. All you need is the data.

In [ ]:
def get_good_portfolios(portfolios):
    good_portfolios = {}
    for i in range(len(portfolios)):
        if portfolios['Portfolio {0}'.format(i)]['Sharpe Ratio'] > 2:
            good_portfolios.update(portfolios['Portfolio {0}'.format(i)]['Weights'])
    return good_portfolios

In [ ]:
good_portfolios = get_good_portfolios(eigen_portfolios)

In [ ]:
stuff = []
for i in range(len(eigen_portfolios)):
    poop = list(eigen_portfolios['Portfolio {0}'.format(i)]['Weights'].index)
    for poo in poop:
        stuff.append(poo)

In [ ]:
good_portfolios

In [ ]:
pd.Series(good_portfolios)

In [ ]:
pd.Series(stuff).value_counts()

In [ ]:
eigen_portfolios['Portfolio 95']['Returns']

In [ ]:
pca.score(pca.components_)